In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.functional as F 
import torch.nn as nn 
import torch 

# All skl imports go here
from sklearn import tree   # Decision Trees
from sklearn import svm    # svm
from sklearn.decomposition import PCA

import sklearn as skl

# Data Loading

In [42]:
# Load all data
train_data = pd.read_csv("training_labse.csv")
test_data = pd.read_csv("test_labse.csv")
validation_data = pd.read_csv("validation_labse.csv")

# Separate X's and y's from each other
FEATURE_COLUMNS = [x for x in train_data if x.startswith("_e")]
LABEL_COLUMN = "label"

X_train = train_data[FEATURE_COLUMNS]
Y_train = train_data[LABEL_COLUMN]

X_test = test_data[FEATURE_COLUMNS]
Y_test = test_data[LABEL_COLUMN]

X_val = validation_data[FEATURE_COLUMNS]
Y_val = validation_data[LABEL_COLUMN]

# These are used to run cross validation
X_train_val = pd.concat([X_train, X_val]) 
Y_train_val = pd.concat([Y_train + Y_test])

In [61]:
# Perform pre-processing PCA on the training set

pca = PCA(
    n_components= 100, 
)

# Need to standardize the data frirst
standardized = (X_train - X_train.mean(axis=0)) / X_train.std(axis = 0)
 
pca.fit(X=standardized)

X_train_d = pca.transform(X=standardized)

[[-11.90018761   3.64564885   2.04592155 ...   0.4870519   -2.49616437
    0.04113879]
 [  2.8493673    2.72139141   5.7759445  ...  -0.31386318  -2.96803635
   -0.16628874]
 [ -6.78434274  -1.67105413   5.41836576 ...   0.31008264  -1.63390021
    0.47133654]
 ...
 [ -9.25338953   6.23527561  -6.66801328 ...   0.23466948  -0.22463135
   -0.17732916]
 [ -2.12141572  -2.59221981   2.05284892 ...  -1.18109567  -1.62549215
    2.66408487]
 [ -0.6367609    0.76828947   7.64508063 ...  -0.54198235  -0.8817055
    0.67446318]]


In [53]:
x = np.array([[1, 2, 3], [4, 5, 6]])
print((x - x.mean(axis = 0)))
print(x.std(axis = 0))

[[-1.5 -1.5 -1.5]
 [ 1.5  1.5  1.5]]
[1.5 1.5 1.5]


# Decision Trees

In [7]:
decision_tree_model = tree.DecisionTreeClassifier(
    criterion='entropy', 
    splitter = 'best',
    max_depth = 5
)


In [ ]:
decision_tree_model.fit(X_train, Y_train)
tree.plot_tree(decision_tree_model)

# SVMs

# MLP

# RNN

# Encoder-Only Transformer

# Test

In [ ]:
model_to_test = decision_tree_model

In [ ]:
#TODO: Add stuff for running the test set on the model